# The Battle of Neighborhoods for a Hungry Homesick Indian Soul

<h5> This notebook uses Foursquare API and Geopy Data to segment the cities of NewYork and Ontario into Clusters based on the presence of Indian Restuarants and recommend the neighborhood clusters best suited to visit for a hungry homesick Indian soul. It also lists the top 10 Indian Restaurants in each city </h5>

## Download Dependencies

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.



## Define Foursquare Credentials and Version

In [2]:
CLIENT_ID = 'DR5VEN3NPEJSSDZE0CFJ42HDTG3G03LBMFE0TB5HKDBCUORY' # your Foursquare ID
CLIENT_SECRET = 'HZMGVQJCQ12CIXFYZ13PM1A5G51S3NNNYSAJJHJRAJQFVD22' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DR5VEN3NPEJSSDZE0CFJ42HDTG3G03LBMFE0TB5HKDBCUORY
CLIENT_SECRET:HZMGVQJCQ12CIXFYZ13PM1A5G51S3NNNYSAJJHJRAJQFVD22


# NewYork City

## N1 Download and Explore Dataset

Newyork has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 


#### N1.1 Download the Newyork Data as shared from the Coursera course

In [3]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


####  N1.2 Load and explore the data

##### N1.2.1 Load the data

In [4]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

##### N1.2.2 Take a look at the data

In [5]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

##### N1.2.3 Define a new variable, feature

In [6]:
neighborhoods_data = newyork_data['features']

##### N1.2.4 take a look at the first item in this list.

In [7]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### N1.3 Tranform the data into a *pandas* dataframe

##### N1.3.1 Transform the data into a *pandas* dataframe

In [8]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

##### N1.3.2 Take a look at the empty dataframe to confirm that the columns are as intended.

In [9]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


##### N1.3.3 Loop through the data and fill the dataframe one row at a time.

In [10]:

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)


Quickly examine the resulting dataframe.

In [11]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [12]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### N1.4 Use geopy library to get the latitude and longitude values of New York City.

In [13]:
address = 'New York City, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of New York City are 40.7308619, -73.9871558.


In [14]:
nyc_data = neighborhoods.reset_index(drop=True)
nyc_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


#### N1.5 Create a map of New York with neighborhoods superimposed on top.

In [15]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

## 2. Explore Neighborhoods in NewYork City

#### Let's create a function to query the top 10 Indian Restuarants within a radius of 500m in all the neighborhoods in Newyork City

In [16]:
def getNearbyVenues(names, latitudes, longitudes, search_query, radius=500):
    
    venues_list=[]
    LIMIT=10
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            search_query,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe

In [17]:
# type your answer here
nyc_venues = getNearbyVenues(names=nyc_data['Neighborhood'],
                                   latitudes=nyc_data['Latitude'],
                                   longitudes=nyc_data['Longitude'],
                                   search_query='Indian'
                                  )


Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

Double-click __here__ for the solution.
<!-- The correct answer is:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
--> 

#### Let's check the size of the resulting dataframe

In [18]:
print(nyc_venues.shape)
nyc_venues.head()

(381, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Kingsbridge,40.881687,-73.902818,Tazmohol Indian Restaurant,40.879331,-73.903192,Indian Restaurant
1,Woodlawn,40.898273,-73.867315,Curry Spot,40.897625,-73.867147,Indian Restaurant
2,Norwood,40.877224,-73.879391,Curry & Kebab,40.874110,-73.878570,Indian Restaurant
3,Parkchester,40.837938,-73.856003,Neerob,40.836111,-73.854902,Indian Restaurant
4,Parkchester,40.837938,-73.856003,Naan Shack,40.835705,-73.855205,Indian Restaurant


Let's check how many venues were returned for each neighborhood

In [19]:
nyc_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,1,1,1,1,1,1
Astoria,6,6,6,6,6,6
Bath Beach,1,1,1,1,1,1
Battery Park City,2,2,2,2,2,2
Bay Ridge,1,1,1,1,1,1
Bayside,4,4,4,4,4,4
Bellaire,1,1,1,1,1,1
Bellerose,1,1,1,1,1,1
Boerum Hill,3,3,3,3,3,3


#### Let's find out how many unique categories can be curated from all the returned venues

In [20]:
print('There are {} uniques categories.'.format(len(nyc_venues['Venue Category'].unique())))

There are 31 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [21]:
# one hot encoding
nyc_onehot = pd.get_dummies(nyc_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nyc_onehot['Neighborhood'] = nyc_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [nyc_onehot.columns[-1]] + list(nyc_onehot.columns[:-1])
nyc_onehot = nyc_onehot[fixed_columns]

nyc_onehot.head()

,Neighborhood,Asian Restaurant,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega,Diner,Donut Shop,Dosa Place,Fast Food Restaurant,Food & Drink Shop,Food Truck,Fried Chicken Joint,Grocery Store,Hotel,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Lounge,Malay Restaurant,Middle Eastern Restaurant,North Indian Restaurant,Pakistani Restaurant,Pizza Place,Sandwich Place,Seafood Restaurant,South Indian Restaurant,Supermarket,Vegetarian / Vegan Restaurant
0,Kingsbridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,Woodlawn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,Norwood,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,Parkchester,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,Parkchester,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [22]:
nyc_onehot.shape

(381, 32)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [23]:
nyc_grouped = nyc_onehot.groupby('Neighborhood').mean().reset_index()
nyc_grouped

,Neighborhood,Asian Restaurant,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega,Diner,Donut Shop,Dosa Place,Fast Food Restaurant,Food & Drink Shop,Food Truck,Fried Chicken Joint,Grocery Store,Hotel,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Lounge,Malay Restaurant,Middle Eastern Restaurant,North Indian Restaurant,Pakistani Restaurant,Pizza Place,Sandwich Place,Seafood Restaurant,South Indian Restaurant,Supermarket,Vegetarian / Vegan Restaurant
0,Allerton,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000,0.0,1.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,Astoria,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,1.000000,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,Bath Beach,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,1.000000,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,Battery Park City,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,1.000000,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,Bay Ridge,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,1.000000,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,Bayside,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,1.000000,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
6,Bellaire,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,1.000000,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
7,Bellerose,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,1.000000,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
8,Boerum Hill,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,1.000000,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
9,Briarwood,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,1.000000,0.0,0.0,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


#### Let's confirm the new size

In [24]:
nyc_grouped.shape

(110, 32)

#### Let's print each neighborhood along with the top 5 most common venues

In [25]:
num_top_venues = 5

for hood in nyc_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = nyc_grouped[nyc_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
                     venue  freq
0      Fried Chicken Joint   1.0
1         Asian Restaurant   0.0
2                    Hotel   0.0
3              Supermarket   0.0
4  South Indian Restaurant   0.0


----Astoria----
                     venue  freq
0        Indian Restaurant   1.0
1         Asian Restaurant   0.0
2                    Hotel   0.0
3              Supermarket   0.0
4  South Indian Restaurant   0.0


----Bath Beach----
                     venue  freq
0        Indian Restaurant   1.0
1         Asian Restaurant   0.0
2                    Hotel   0.0
3              Supermarket   0.0
4  South Indian Restaurant   0.0


----Battery Park City----
                     venue  freq
0        Indian Restaurant   1.0
1         Asian Restaurant   0.0
2                    Hotel   0.0
3              Supermarket   0.0
4  South Indian Restaurant   0.0


----Bay Ridge----
                     venue  freq
0        Indian Restaurant   1.0
1         Asian Restaurant   0.0
2    

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = nyc_grouped['Neighborhood']

for ind in np.arange(nyc_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nyc_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Fried Chicken Joint,Vegetarian / Vegan Restaurant,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega,Diner
1,Astoria,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
2,Bath Beach,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
3,Battery Park City,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
4,Bay Ridge,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
5,Bayside,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
6,Bellaire,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
7,Bellerose,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
8,Boerum Hill,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
9,Briarwood,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [28]:
# set number of clusters
kclusters = 3

nyc_grouped_clustering = nyc_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nyc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [29]:
print(nyc_data.shape)
print(neighborhoods_venues_sorted.shape)

(306, 4)
(110, 11)


In [30]:
n1=nyc_data['Neighborhood'].tolist()
n2=neighborhoods_venues_sorted['Neighborhood'].tolist()
result=set(n1).difference(n2)

index_a=[]
for i in result:
    index_a.append(n1.index(i))
nyc_data_new=nyc_data.drop(nyc_data.index[index_a])


n1=nyc_data_new['Neighborhood'].tolist()
n2=neighborhoods_venues_sorted['Neighborhood'].tolist()
result=set(n1).difference(n2)
index_a=[]
for i in result:
    index_a.append(n1.index(i))
nyc_data_new1=nyc_data_new.drop(nyc_data_new.index[index_a])

print(n1.index('Chelsea'))
print(n1.index('Murray Hill'))
print(n1.index('Sunnyside'))
print(n1.index('Bay Terrace'))

nyc_data_new1=nyc_data_new.drop(nyc_data_new.index[[45,44,65,93]])
print(nyc_data_new1.shape)

45
44
65
93
(110, 4)


In [31]:
nyc_merged = nyc_data_new1

# add clustering labels
nyc_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
nyc_merged = nyc_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

nyc_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Bronx,Kingsbridge,40.881687,-73.902818,2,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
7,Bronx,Woodlawn,40.898273,-73.867315,1,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
8,Bronx,Norwood,40.877224,-73.879391,1,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
30,Bronx,Parkchester,40.837938,-73.856003,1,Indian Restaurant,South Indian Restaurant,Supermarket,Indian Chinese Restaurant,Vegetarian / Vegan Restaurant,Deli / Bodega,Dosa Place,Donut Shop,Diner,Coffee Shop
38,Bronx,Schuylerville,40.826580,-73.826203,1,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega


Finally, let's visualize the resulting clusters

In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nyc_merged['Latitude'], nyc_merged['Longitude'],nyc_merged['Neighborhood'], nyc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [33]:
nyc_merged.loc[nyc_merged['Cluster Labels'] == 0, nyc_merged.columns[[1] + list(range(5,nyc_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,Kensington,Indian Restaurant,Indian Chinese Restaurant,Vegetarian / Vegan Restaurant,Food Truck,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop
59,Prospect Heights,Indian Restaurant,North Indian Restaurant,Vegetarian / Vegan Restaurant,Food Truck,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop
68,Gowanus,Grocery Store,Indian Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
70,Park Slope,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
84,Clinton Hill,Indian Restaurant,Cocktail Bar,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Coffee Shop,Deli / Bodega
96,North Side,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
97,South Side,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
98,Ocean Parkway,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
101,Washington Heights,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
105,Central Harlem,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega


#### Cluster 2

In [34]:
must_visit=nyc_merged.loc[nyc_merged['Cluster Labels'] == 1, nyc_merged.columns[[1] + list(range(5, nyc_merged.shape[1]))]]
nyc_merged.loc[nyc_merged['Cluster Labels'] == 1, nyc_merged.columns[[1] + list(range(5, nyc_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Woodlawn,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
8,Norwood,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
30,Parkchester,Indian Restaurant,South Indian Restaurant,Supermarket,Indian Chinese Restaurant,Vegetarian / Vegan Restaurant,Deli / Bodega,Dosa Place,Donut Shop,Diner,Coffee Shop
38,Schuylerville,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
44,Unionport,Donut Shop,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop
46,Bay Ridge,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
49,Greenpoint,Vegetarian / Vegan Restaurant,Indian Restaurant,Grocery Store,Seafood Restaurant,Sandwich Place,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar
52,Sheepshead Bay,Grocery Store,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega,Diner
54,Flatbush,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
58,Windsor Terrace,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega


#### Cluster 3

In [35]:
nyc_merged.loc[nyc_merged['Cluster Labels'] == 2, nyc_merged.columns[[1] + list(range(5, nyc_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Kingsbridge,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
66,Carroll Gardens,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
90,City Line,Indian Restaurant,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop
107,Upper East Side,Vegetarian / Vegan Restaurant,Seafood Restaurant,Indian Restaurant,Food Truck,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop
129,Astoria,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
138,Flushing,Malay Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
220,Sunnyside,Indian Restaurant,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Bar,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega
273,Turtle Bay,Indian Restaurant,Bar,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Breakfast Spot,Brewery,Café,Cocktail Bar,Coffee Shop,Deli / Bodega


<h1> Acccording to the Analysis in NYC,
    <ul>
        <li><b>Cluster 2</b> has the neighborhoods for <b>Must Visit for Hungry Home Sick Indian Soul</b></li>
        <li><b>Cluster 1</b> has the neighborhoods for <b>Can Visit for Hungry Home Sick Indian Soul</b></li>
        <li><b>Cluster 3</b> has the neighborhoods for <b>Should not Visit for Hungry Home Sick Indian Soul</b></li>
    </ul>
</h1>

# <b> TOP 10 Indian Restaurants in Newyork City </b>

In [36]:
address = "Newyork, NY"
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Neighborhood are {}, {}.'.format(latitude, longitude))
LIMIT = 10 # limit of number of venues returned by Foursquare API
search_query = "Indian"
radius = 5000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    search_query,
    radius, 
    LIMIT)
results = requests.get(url).json()["response"]['groups'][0]['items']
venues_list=[]
    # return only relevant information for each nearby venue
venues_list.append([(
            v['venue']['name'], 
            v['venue']['location']['address'],  
            v['venue']['categories'][0]['name']) for v in results])

nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = [
                  'Venue', 
                  'Address',
                  'Venue Category']
    
nearby_venues


/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


The geograpical coordinate of Neighborhood are 40.83975585, -73.9414480148711.


,Venue,Address,Venue Category
0,Kismat Indian Restaurant,603 Fort Washington Ave,Indian Restaurant
1,Chapati House - NYC,3153 Broadway,Indian Restaurant
2,Mumbai Masala,1768 Amsterdam Ave,Indian Restaurant
3,Hungry Bird,980 Morris Ave,Indian Restaurant
4,Clove Indian Restaurant & Bar,1592 Amsterdam Ave,Indian Restaurant
5,Delhi Masala,2077 Adam Clayton Powell Jr Blvd,Indian Restaurant
6,Roti Roll,994 Amsterdam Ave,Indian Restaurant
7,Kinara Cuisine of India,880 River Rd,Indian Restaurant
8,Doaba Deli,945 Columbus Ave,Indian Restaurant
9,Maharaja Palace,2113 Frederick Douglass Blvd,Indian Restaurant


# TORONTO

## 1. Download and Explore the Dataset

<p> Since the dataset for Toronto is not readily available, it is required to download and wrangle the dataset from wikipedia </p>

### Read content of the wikipedia page containing the data about Toronto

In [37]:
# Get data from the wikipedia page
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto_data_wiki=requests.get(url)
# Convert the data to string and display the data
toronto_wiki_page=toronto_data_wiki.text

### Wrangle the data

<p> The data read from the wikipedia page contains a table which needs to be converted to a pandas dataframe. This can be done with the help of the BeautifulSoup package </p>

In [38]:
# Install beautiful soup
!conda install -c anaconda beautifulsoup4 --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.6.1                |           py36_0         1.7 MB  anaconda
    ------------------------------------------------------------
                                           Total:         1.7 MB

The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2018.11.~ --> anaconda::ca-certificates-2018.12.5-0
  cryptography       conda-forge::cryptography-2.3.1-py36h~ --> anaconda::cryptography-2.4.2-py36h1ba5d50_0
  grpcio             conda-forge::grpcio-1.16.0-py36h4f00d~ --> anaconda::grpcio-1.16.1-py36hf8bcb03_1
  openssl            conda-forge::openssl-1.0.2p-h14c3975_~ --> anaconda::openssl-1.1.1-h7b6447c_0
  python             conda-forge::python-3.6.6-hd21bae

In [39]:
# Import the BeautifulSoup Package 
from bs4 import BeautifulSoup
bs = BeautifulSoup(toronto_wiki_page,'lxml')
# Use prettify function to determine the html table class name which needs to be extracted
#print(bs.prettify())

In [40]:
# Extract the table from the page
toronto_table = bs.find('table',{'class':'wikitable sortable'})
# toronto_table // Display the table

In [41]:
#Extract postcode, borough and neighbour hood lists based on conditions specified
rows = toronto_table.find_all('tr')
postcode=[]
borough=[]
neighbourhood=[]
for row in rows:
    cols=row.find_all('td')
    cols=[x.text.strip() for x in cols] # Cols is a list with 3 elements in the order postcode, borough,neighbourhood
    if cols:
        if(cols[1] !='Not assigned'): # Ignore cells with a borough that is Not assigned.
            borough.append(cols[1]) 
            if(cols[2] == 'Not assigned'): # If neighborhood is not assigned, it is the same as borough
                neighbourhood.append(cols[1])
            else:
                neighbourhood.append(cols[2])
            if(cols[0] != 'Not Assigned'):
                postcode.append(cols[0]) 

# Display lists
#print(postcode)
#print(borough)
#print(neighbourhood)

In [42]:
# Combine the lists to a dataframe
toronto_df_ini=pd.DataFrame(
    {'Postalcode': postcode,
     'Borough': borough,
     'Neighborhood': neighbourhood
    })
toronto_df_ini.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


<p> Concatenate the neighborhoods coming under the same postal code using lambda functions and pandas groupby function</p>

In [43]:
def concatenate_neighborhood(x):
    neigh_concat = ""
    for i in range(len(x)-1):
        neigh_concat = neigh_concat + x.iloc[i] + ", "
    neigh_concat += x.iloc[-1]
    return neigh_concat

def select_Borough(x):
    borough_sel = x.iloc[0]
    for i in range(1, len(x)):
        if borough_sel != x.iloc[i]:
            for i in x:
                print(x)
            raise Exception("Postcode comprises two Boroughs")
    return borough_sel
toronto_df = toronto_df_ini.groupby(["Postalcode"]).agg({"Borough": lambda x: select_Borough(x),
                                 "Neighborhood": lambda x: concatenate_neighborhood(x)},as_index=False)
toronto_df = toronto_df.reset_index()
toronto_df.head(5)

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Display Shape of dataframe

In [44]:
toronto_df.shape

(103, 3)

## 2. GeoSpatial Analysis

### Download csv file

<p> Since the geocoder package is not reliable enough, the csv file provided by coursera is used for the latitude and longitude coordinated of locations </p>

In [45]:
# Download the csv file
!wget -q -O 'lat_long_data.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [46]:
# Read the csv data into a pandas data frame
lat_long_df = pd.read_csv('lat_long_data.csv')
lat_long_df.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [47]:
# Remove postal code column from lat_long_df before concatenation
lat_long_df.drop(['Postal Code'],axis=1,inplace=True)
lat_long_df.head()

,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476


In [48]:
# Concatentate the two data frames
toronto_df_geo=pd.concat([toronto_df,lat_long_df],axis=1,sort=False)
toronto_df_geo.head(5)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## 3. Neighborhood Analysis in Toronto

#### Print Boroughs and neighborhood information in Toronto

In [49]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_df_geo['Borough'].unique()),
        toronto_df_geo.shape[0]
    )
     )

The dataframe has 11 boroughs and 103 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto City

In [50]:
address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top

<p> The Folium Library is used to create the map </p>

In [51]:
# create map of New York using latitude and longitude values
map_toronto= folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_geo['Latitude'], toronto_df_geo['Longitude'], toronto_df_geo['Borough'], toronto_df_geo['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Black',
        fill=True,
        fill_color='#9400D3',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## 4. Explore all neighborhoods in Toronto

####  The getnearbyvalues function is used to explore and get the values of all the neighborhoods using loops 

In [54]:
CLIENT_ID = 'DR5VEN3NPEJSSDZE0CFJ42HDTG3G03LBMFE0TB5HKDBCUORY' # your Foursquare ID
CLIENT_SECRET = 'Z4EAEP4L50TAR4HSZPG1P35DI4X3CO0PJSBDXZCNUPABXYGT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DR5VEN3NPEJSSDZE0CFJ42HDTG3G03LBMFE0TB5HKDBCUORY
CLIENT_SECRET:Z4EAEP4L50TAR4HSZPG1P35DI4X3CO0PJSBDXZCNUPABXYGT


In [59]:
def getNearbyVenues(names, latitudes, longitudes, search_query, radius=400):
    
    venues_list=[]
    LIMIT=10
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            search_query,
            radius, 
            LIMIT)
              # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)      
        


#### Create a new dataframe called toronto_venues containing the neighborhood analysis for entire Toronto City

In [60]:
toronto_venues = getNearbyVenues(names=toronto_df_geo['Neighborhood'],
                                   latitudes=toronto_df_geo['Latitude'],
                                   longitudes=toronto_df_geo['Longitude'],
                                   search_query='Indian'
                                  )


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The D

#### Shape of Toronto Venues

In [61]:
toronto_venues.shape
toronto_venues.head()
toronto_venues = toronto_venues.reset_index()

#### Count of Venues for each neighborhood

In [62]:
toronto_venues.groupby('Neighborhood').count()

,index,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
"Adelaide, King, Richmond",4,4,4,4,4,4,4
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",2,2,2,2,2,2,2
"Alderwood, Long Branch",1,1,1,1,1,1,1
"Bedford Park, Lawrence Manor East",2,2,2,2,2,2,2
"Brockton, Exhibition Place, Parkdale Village",1,1,1,1,1,1,1
"Cabbagetown, St. James Town",3,3,3,3,3,3,3
Cedarbrae,1,1,1,1,1,1,1
Central Bay Street,5,5,5,5,5,5,5
"Chinatown, Grange Park, Kensington Market",3,3,3,3,3,3,3


#### Unique categories curated from all the returned venues

In [63]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 11 uniques categories.


## 5. Analyze each Neighborhood in Toronto

In [64]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True)
toronto_onehot.insert(0, 'Neighborhood', toronto_venues['Neighborhood'])
toronto_onehot.head()

,Neighborhood,Bar,Chinese Restaurant,Deli / Bodega,Diner,Event Space,Grocery Store,Indian Restaurant,Market,Middle Eastern Restaurant,Restaurant,South Indian Restaurant
0,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,1,0,0,0,0
1,Cedarbrae,0,0,0,0,0,0,0,0,0,0,1
2,"Dorset Park, Scarborough Town Centre, Wexford ...",0,0,0,0,0,0,1,0,0,0,0
3,"Maryvale, Wexford",0,0,0,0,0,0,1,0,0,0,0
4,"L'Amoreaux West, Steeles West",0,0,0,0,0,0,1,0,0,0,0


#### Size of the one hot encoded data frame

In [65]:
toronto_onehot.shape

(77, 12)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [66]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Bar,Chinese Restaurant,Deli / Bodega,Diner,Event Space,Grocery Store,Indian Restaurant,Market,Middle Eastern Restaurant,Restaurant,South Indian Restaurant
0,"Adelaide, King, Richmond",0.250000,0.000000,0.000000,0.000000,0.000000,0.00,0.750000,0.000000,0.00,0.000000,0.0
1,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,1.000000,0.000000,0.00,0.000000,0.0
2,"Alderwood, Long Branch",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,1.000000,0.000000,0.00,0.000000,0.0
3,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,1.000000,0.000000,0.00,0.000000,0.0
4,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,1.000000,0.000000,0.00,0.000000,0.0
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.666667,0.333333,0.00,0.000000,0.0
6,Cedarbrae,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,1.0
7,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,1.000000,0.000000,0.00,0.000000,0.0
8,"Chinatown, Grange Park, Kensington Market",0.000000,0.333333,0.000000,0.000000,0.000000,0.00,0.333333,0.000000,0.00,0.333333,0.0
9,Church and Wellesley,0.000000,0.000000,0.000000,0.333333,0.000000,0.00,0.666667,0.000000,0.00,0.000000,0.0


#### Confirm the new size

In [67]:
toronto_grouped.shape

(32, 12)

####  Print each neighborhood along with the top 5 most common venues

In [68]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                venue  freq
0   Indian Restaurant  0.75
1                 Bar  0.25
2  Chinese Restaurant  0.00
3       Deli / Bodega  0.00
4               Diner  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                venue  freq
0   Indian Restaurant   1.0
1                 Bar   0.0
2  Chinese Restaurant   0.0
3       Deli / Bodega   0.0
4               Diner   0.0


----Alderwood, Long Branch----
                venue  freq
0   Indian Restaurant   1.0
1                 Bar   0.0
2  Chinese Restaurant   0.0
3       Deli / Bodega   0.0
4               Diner   0.0


----Bedford Park, Lawrence Manor East----
                venue  freq
0   Indian Restaurant   1.0
1                 Bar   0.0
2  Chinese Restaurant   0.0
3       Deli / Bodega   0.0
4               Diner   0.0


----Brockton, Exhibition Place, Parkdale Village----
                venue  freq
0   Indian Rest

####  Create a Pandas Dataframe with the above values

In [69]:
# Sort Venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [70]:
# Create new dataframe
num_top_venues = 9

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue
0,"Adelaide, King, Richmond",Indian Restaurant,Bar,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Diner
1,"Albion Gardens, Beaumond Heights, Humbergate, ...",Indian Restaurant,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Diner,Deli / Bodega
2,"Alderwood, Long Branch",Indian Restaurant,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Diner,Deli / Bodega
3,"Bedford Park, Lawrence Manor East",Indian Restaurant,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Diner,Deli / Bodega
4,"Brockton, Exhibition Place, Parkdale Village",Indian Restaurant,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Diner,Deli / Bodega
5,"Cabbagetown, St. James Town",Indian Restaurant,Market,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Grocery Store,Event Space,Diner,Deli / Bodega
6,Cedarbrae,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Indian Restaurant,Grocery Store,Event Space,Diner,Deli / Bodega
7,Central Bay Street,Indian Restaurant,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Diner,Deli / Bodega
8,"Chinatown, Grange Park, Kensington Market",Restaurant,Indian Restaurant,Chinese Restaurant,South Indian Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Diner
9,Church and Wellesley,Indian Restaurant,Diner,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Deli / Bodega


## 6. Cluster the neighborhoods

#### Run k-means to cluster the neighborhood into 7 clusters

In [71]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 1, 1, 1, 0, 2, 1, 0, 0], dtype=int32)

#### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

<h3><i> Optional Step since there was an index mismatch for my two arrays, I checked the difference and remove the addditional rows</i></h3>

In [72]:
n1=toronto_df_geo['Neighborhood'].tolist()
n2=neighborhoods_venues_sorted['Neighborhood'].tolist()
result=set(n1).difference(n2)

index_a=[]
for i in result:
    index_a.append(n1.index(i))
toronto_df_new=toronto_df_geo.drop(toronto_df_geo.index[index_a])

<h3></i> Drop the rows </i> </h3>

In [73]:

toronto_merged=toronto_df_new
# add clustering labels

toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Indian Restaurant,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Diner,Deli / Bodega
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Indian Restaurant,Grocery Store,Event Space,Diner,Deli / Bodega
10,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,1,Indian Restaurant,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Diner,Deli / Bodega
11,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849,1,Indian Restaurant,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Diner,Deli / Bodega
15,M1W,Scarborough,"L'Amoreaux West, Steeles West",43.799525,-79.318389,1,Indian Restaurant,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Diner,Deli / Bodega


### Visualize the resulting clusters

In [74]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 7. Examine Clusters

### Cluster 1

In [75]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue
2,Scarborough,0,Indian Restaurant,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Diner,Deli / Bodega
18,North York,0,Indian Restaurant,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Diner,Deli / Bodega
41,East Toronto,0,Indian Restaurant,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Diner,Deli / Bodega
42,East Toronto,0,Indian Restaurant,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Diner,Deli / Bodega
47,Central Toronto,0,Indian Restaurant,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Diner,Deli / Bodega
52,Downtown Toronto,0,Indian Restaurant,Diner,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Deli / Bodega
57,Downtown Toronto,0,Indian Restaurant,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Diner,Deli / Bodega
72,North York,0,Indian Restaurant,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Diner,Deli / Bodega


In [76]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue
4,Scarborough,1,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Indian Restaurant,Grocery Store,Event Space,Diner,Deli / Bodega
10,Scarborough,1,Indian Restaurant,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Diner,Deli / Bodega
11,Scarborough,1,Indian Restaurant,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Diner,Deli / Bodega
15,Scarborough,1,Indian Restaurant,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Diner,Deli / Bodega
39,East York,1,Indian Restaurant,Grocery Store,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Event Space,Diner,Deli / Bodega
51,Downtown Toronto,1,Indian Restaurant,Market,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Grocery Store,Event Space,Diner,Deli / Bodega
54,Downtown Toronto,1,Indian Restaurant,Middle Eastern Restaurant,South Indian Restaurant,Restaurant,Market,Grocery Store,Event Space,Diner,Deli / Bodega
55,Downtown Toronto,1,Indian Restaurant,Event Space,Diner,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Deli / Bodega
58,Downtown Toronto,1,Indian Restaurant,Bar,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Diner
59,Downtown Toronto,1,Indian Restaurant,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Diner,Deli / Bodega


In [77]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue
35,East York,2,Indian Restaurant,South Indian Restaurant,Restaurant,Middle Eastern Restaurant,Market,Grocery Store,Event Space,Diner,Deli / Bodega


<h1> Acccording to the Analysis in Toronto,
    <ul>
        <li><b>Cluster 2</b> has the neighborhoods for <b>Must Visit for Hungry Home Sick Indian Soul</b></li>
        <li><b>Cluster 1</b> has the neighborhoods for <b>Can Visit for Hungry Home Sick Indian Soul</b></li>
        <li><b>Cluster 3</b> has the neighborhoods for <b>Should not Visit for Hungry Home Sick Indian Soul</b></li>
    </ul>
</h1>

# <b> TOP 10 Indian Restaurants in Toronto </b>

In [78]:
address = "Toronto,ON"
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Neighborhood are {}, {}.'.format(latitude, longitude))
LIMIT = 10 # limit of number of venues returned by Foursquare API
search_query = "Indian"
radius = 5000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    search_query,
    radius, 
    LIMIT)
results = requests.get(url).json()["response"]['groups'][0]['items']
venues_list=[]
    # return only relevant information for each nearby venue
venues_list.append([(
            v['venue']['name'], 
            v['venue']['location']['address'],  
            v['venue']['categories'][0]['name']) for v in results])

nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = [
                  'Venue', 
                  'Address',
                  'Venue Category']
    
nearby_venues


/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


The geograpical coordinate of Neighborhood are 43.653963, -79.387207.


,Venue,Address,Venue Category
0,Little India Restaurant,255 Queen St. W,Indian Restaurant
1,Banjara Indian Cuisine,796 Bloor St W,Indian Restaurant
2,Kothur Indian Cuisine,649 Yonge St.,Indian Restaurant
3,Gandhi Cuisine,554 Queen St. W.,Indian Restaurant
4,Butter Chicken Factory,556 Parliament St.,Indian Restaurant
5,Utsav Indian Cuisine,69 Yorkville Ave.,Indian Restaurant
6,Aroma Fine Indian Restaurant,287 King St. W,Indian Restaurant
7,King's Palace,820 Church St.,Indian Restaurant
8,Bombay Palace,71 Jarvis St,Indian Restaurant
9,Bombay Street Food,828 Bay Street,Indian Restaurant


<h1>************************************** THANK YOU ************************************** </h2>